In [1]:
# conda install conda-forge::openjdk

In [2]:
# conda install anaconda::pyspark

In [3]:
# conda install conda-forge::findspark

In [4]:
# export SPARK_HOME="/Downloads/spark"
# export PATH=$SPARK_HOME/bin:$PATH
# export PYSPARK_PYTHON=python3

In [5]:
import findspark
findspark.init()

import pyspark
findspark.find()

'/opt/anaconda3/lib/python3.11/site-packages/pyspark'

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext()
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/11 20:23:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file = "kddcup.data_10_percent.gz"
data = sc.textFile(file)

In [4]:
print("Lines: ", data.count())

Lines:  494021


25/07/11 20:24:10 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
data.take(3)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.']

In [6]:
data.first()

'0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.'

In [7]:
normal_data = data.filter(lambda x: 'normal.' in x)

In [11]:
from time import time
t0 = time()
normal_count = normal_data.count()
t1 = time() - t0
print("There are", normal_count, "'normal's.")
print("Time to count:", format(round(t1, 3)), "seconds")

There are 97278 'normal's.
Time to count: 0.295 seconds


In [12]:
t0 = time()
array_data = data.collect()
t1 = time() - t0
print("Data collected in", round(t1, 3), "seconds")

Data collected in 0.869 seconds


In [13]:
data_sample = data.sample(False, 0.1, 42)
sample_size = data_sample.count()
total_size = data.count()
print("Sample size is", sample_size, "of", total_size)

Sample size is 49387 of 494021


In [14]:
sample_normal_tags = data_sample.filter(lambda x: "normal." in x)

t0 = time()
sample_normal_tags_count = sample_normal_tags.count()
t1 = time() - t0

sample_normal_ratio = sample_normal_tags_count / float(sample_size)

print("Sample with normal size is", sample_normal_tags_count)
print("The ratio of 'normal' interactions is ", round(sample_normal_ratio, 3))
print("Count done in", round(t1, 3), "seconds")

Sample with normal size is 9733
The ratio of 'normal' interactions is  0.197
Count done in 0.266 seconds


In [15]:
sample_without_normal = data_sample.subtract(sample_normal_tags)
print("Sample without normal size is", sample_without_normal.count())

Sample without normal size is 39654


In [16]:
sample_without_normal.take(1)

['0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,233,0.91,0.01,0.91,0.00,0.00,0.00,0.00,0.00,smurf.']

In [17]:
array_RDD = data.map(lambda x: x.split(","))

In [18]:
array_RDD.first()

['0',
 'tcp',
 'http',
 'SF',
 '181',
 '5450',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '8',
 '8',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '1.00',
 '0.00',
 '0.00',
 '9',
 '9',
 '1.00',
 '0.00',
 '0.11',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 'normal.']

In [21]:
protocols = array_RDD.map(lambda x: x[1]).distinct()
protocols.collect()

['tcp', 'udp', 'icmp']

In [22]:
print("Protocol:", protocols.count())

Protocol: 3


In [23]:
services = array_RDD.map(lambda x: x[2]).distinct()
services.collect()

['http',
 'smtp',
 'finger',
 'domain_u',
 'auth',
 'telnet',
 'ftp',
 'eco_i',
 'ntp_u',
 'ecr_i',
 'other',
 'private',
 'pop_3',
 'ftp_data',
 'rje',
 'time',
 'mtp',
 'link',
 'remote_job',
 'gopher',
 'ssh',
 'name',
 'whois',
 'domain',
 'login',
 'imap4',
 'daytime',
 'ctf',
 'nntp',
 'shell',
 'IRC',
 'nnsp',
 'http_443',
 'exec',
 'printer',
 'efs',
 'courier',
 'uucp',
 'klogin',
 'kshell',
 'echo',
 'discard',
 'systat',
 'supdup',
 'iso_tsap',
 'hostnames',
 'csnet_ns',
 'pop_2',
 'sunrpc',
 'uucp_path',
 'netbios_ns',
 'netbios_ssn',
 'netbios_dgm',
 'sql_net',
 'vmnet',
 'bgp',
 'Z39_50',
 'ldap',
 'netstat',
 'urh_i',
 'X11',
 'urp_i',
 'pm_dump',
 'tftp_u',
 'tim_i',
 'red_i']

In [24]:
print("Service:", services.count())

Service: 66
